In [ ]:
from datetime import datetime
import torch
import requests
import cv2
from PIL import Image, ImageOps
from io import BytesIO
from diffusers import StableDiffusionImg2ImgPipeline, ControlNetModel, StableDiffusionControlNetPipeline
from diffusers.utils import load_image
from controlnet_aux import OpenposeDetector, HEDdetector
import logging
openpose = OpenposeDetector.from_pretrained("lllyasviel/ControlNet")
hed = HEDdetector.from_pretrained('lllyasviel/ControlNet')

# cnmodel = "lllyasviel/sd-controlnet-openpose"
# controlnet = ControlNetModel.from_pretrained(
#     cnmodel,
#     torch_dtype=torch.float16,
#     use_auth_token=TOKEN,
# )
#     ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16),
#     ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-hed", torch_dtype=torch.float16)
ctrlnet1_id = "lllyasviel/sd-controlnet-openpose"
controlnet = ControlNetModel.from_pretrained(ctrlnet1_id, torch_dtype=torch.float16)

device = "cuda"
pipeline_id = "Lykon/DreamShaper"
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    pipeline_id,
    controlnet=controlnet,
    safety_checker=None,
    torch_dtype=torch.float16,
    # use_auth_token=TOKEN,
).to(device)

In [ ]:
log = logging.getLogger(__name__)
log.setLevel(logging.INFO)
log_name = r'./log/ctrl3.log'
logging.basicConfig(filename=log_name)
rh = logging.handlers.RotatingFileHandler(
    log_name, 
    encoding='utf-8',
    maxBytes=10000,
    backupCount=100
)

if not log.hasHandlers():
    log.addHandler(rh)

In [ ]:
pose_path = "pic/pose4.png"
pose = Image.open(pose_path).convert("RGB")
openpose_image = openpose(pose)
# Define the cropping area (left, upper, right, lower)
# crop_area = (100,200,400,700)
# pose = openpose_image.crop(crop_area)
# width, height = pose.size
# scale = 4
# new_w, new_h = width*scale, height*scale
# pose = pose.resize((new_w, new_h), Image.ANTIALIAS)
pose = openpose_image
pose

In [ ]:
prompt = "Spartan warrior, {pure-white-skin}, {ashen-skin}, {ghost-skin}, bald, beard to his chest, muscle, sitting on the rock, upper-naked, red waistcloth, below scarlet sky, Roman shrine shattered, highly detailed, 4k"
negative_prompt = "closeup, (deformed iris, deformed pupils, cgi, render, sketch, cartoon, anime:1.4), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck"
generator = torch.Generator(device=device).manual_seed(1024)
guidance_scale = 8
steps = 50
image = pipe(prompt=prompt, negative_prompt=negative_prompt, image=pose, guidance_scale=guidance_scale, num_inference_steps=steps, generator=generator).images[0]
now = datetime.now()
formatted_time = now.strftime("%Y%m%d%H%M%S")
image.save(f'pic/{formatted_time}.png')

log.info("***")
log.info(formatted_time)
log.info(f"PROMPT: {prompt}")
log.info(f"NEGATIVE PROMPT: {negative_prompt}")
log.info(f"MODEL: {pipeline_id}")
log.info(f"CONTROL NET 1: {ctrlnet1_id}")
log.info(f"IMAGE 1 PATH: {pose_path}")
log.info(f"GUIDANCE SCALE: {guidance_scale}")
log.info(f"STEPS: {steps}")
image